In [1]:
from loadFilteredData import *
#loading all data
origImg = loadAllTopicData('original')
gingham = loadAllTopicData('gingham')
clarendon = loadAllTopicData('clarendon')
juno = loadAllTopicData('juno')
lark = loadAllTopicData('lark')
gotham = loadAllTopicData('gotham')
reyes = loadAllTopicData('reyes')

['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphitheater
Loading Topic: amusement_park
Loading Topic: aquarium
Loading Topic: aqueduct
Loading Topic: art_gallery
Loading Topic: assembly_line
Loading Topic: auditorium
(9000, 128, 128, 3)
['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphitheater
Loading Topic: amusement_park
Loading Topic: aquarium
Loading Topic: aqueduct
Loading Topic: art_gallery
Loading Topic: assembly_line
Loading Topic: auditorium
(9000, 128, 128, 3)
['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphithe

In [2]:
placeLabels = []
for i in range(9):
    for j in range(1000):
        placeLabels.append([i])
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
placeLabels = enc.fit_transform(placeLabels)
placeLabels.shape

C:\Users\Chunlok Lo\AppData\Local\conda\conda\envs\magiclearning\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(9000, 9)

In [3]:
def createDataPlaces(images, places, trainPercentage):
    categories = len(images)

    imgList = []
    vectors = []
    trainPlaces = []

    testImgList = []
    testVectors = []
    testPlaces = []


    #data for original image
    for c in range(categories):
        numImages = images[c].shape[0]
        print(numImages)
        numTrain = int(numImages * trainPercentage)
        imgList.append(images[c][:numTrain])

        featureVector = np.zeros((numTrain, categories))
        featureVector[:, c] = 1
        vectors.append(featureVector)
        
        trainPlaces.append(places[:numTrain])

        #testing data
        testImgList.append(images[c][numTrain:])

        featureVector = np.zeros((numImages - numTrain, categories))
        featureVector[:, c] = 1
        testVectors.append(featureVector)
        
        testPlaces.append(places[numTrain:])


    X = np.vstack(imgList)
    y = np.vstack(vectors)

    Xtest = np.vstack(testImgList)
    ytest = np.vstack(testVectors)
    
    trainPlaces = np.vstack(trainPlaces)
    testPlaces = np.vstack(testPlaces)
    

    X, y = shuffle(X, y, random_state=0)
    Xtest, ytest = shuffle(Xtest, ytest, random_state=0)
    return X, y, Xtest, ytest, trainPlaces, testPlaces
X, y, Xtest, ytest, trainPlaces, testPlaces = createDataPlaces([origImg, clarendon, gingham, juno, lark, gotham, reyes], placeLabels, .8)

9000
9000
9000
9000
9000
9000
9000


In [5]:
def threeChannelHistogram(images, bins):
    histograms = []
    for image in images:
        redHist = np.histogram(image[:, :, 0], bins=bins, range=(0, 255))[0]
        greenHist = np.histogram(image[:, :, 0], bins=bins, range=(0, 255))[0]
        blueHist = np.histogram(image[:, :, 0], bins=bins, range=(0, 255))[0]
        #print(np.hstack((redHist, greenHist, blueHist)))
        histograms.append(np.hstack((redHist, greenHist, blueHist)))
        #histograms.append(np.histogram(image, bins=bins, range=(0, 255))[0])
    return np.stack(histograms)
bins = 250
categories = 7
Xhist = threeChannelHistogram(X, bins)

In [6]:
Xhist.shape

(50400, 750)

In [7]:
np.save('Xhist.npy', Xhist)

In [8]:
Xtest_hist = threeChannelHistogram(Xtest, bins)

In [9]:
np.save('Xhist_test.npy', Xtest_hist)

In [11]:
XhistPlace = np.hstack((Xhist, trainPlaces))

In [12]:
XhistPlace.shape

(50400, 759)

In [13]:
np.save('XhistPlace.npy', XhistPlace)

In [26]:
from preprocessing import *
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(256, input_dim=bins * 3 + 9, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(categories, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [25]:
model.fit(XhistPlace, y, epochs=150, batch_size=256, validation_split=0.01)

Epoch 1/150
50400/50400 [==============================] - 2s 43us/step - loss: 2.1594 - acc: 0.1492
Epoch 2/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.9069 - acc: 0.1842
Epoch 3/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.8602 - acc: 0.2043
Epoch 4/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.7668 - acc: 0.2476
Epoch 5/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.6482 - acc: 0.2839
Epoch 6/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.5573 - acc: 0.3119
Epoch 7/150
50400/50400 [==============================] - 2s 30us/step - loss: 1.4948 - acc: 0.3513
Epoch 8/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.4055 - acc: 0.3883
Epoch 9/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.3645 - acc: 0.4031
Epoch 10/150
50400/50400 [==============================] - 2s 31us/step - loss: 1.3379 - a

50400/50400 [==============================] - 2s 31us/step - loss: 0.5651 - acc: 0.7894
Epoch 82/150
50400/50400 [==============================] - 2s 33us/step - loss: 0.5642 - acc: 0.7897
Epoch 83/150
50400/50400 [==============================] - 2s 36us/step - loss: 0.5582 - acc: 0.7908: 0s - loss: 0.5579 - acc:
Epoch 84/150
50400/50400 [==============================] - 2s 41us/step - loss: 0.5575 - acc: 0.7908
Epoch 85/150
50400/50400 [==============================] - 2s 37us/step - loss: 0.5513 - acc: 0.7947
Epoch 86/150
50400/50400 [==============================] - 2s 36us/step - loss: 0.5543 - acc: 0.7927
Epoch 87/150
50400/50400 [==============================] - 2s 36us/step - loss: 0.5498 - acc: 0.7942
Epoch 88/150
50400/50400 [==============================] - 2s 35us/step - loss: 0.5496 - acc: 0.7938
Epoch 89/150
50400/50400 [==============================] - 2s 39us/step - loss: 0.5481 - acc: 0.7970: 0s - loss: 0.541
Epoch 90/150
50400/50400 [=========================

In [27]:
model.fit(XhistPlace, y, epochs=150, batch_size=256, validation_split=0.01)

Train on 49896 samples, validate on 504 samples
Epoch 1/150
49896/49896 [==============================] - 2s 46us/step - loss: 2.0091 - acc: 0.2287 - val_loss: 1.7518 - val_acc: 0.2381
Epoch 2/150
49896/49896 [==============================] - 2s 33us/step - loss: 1.7543 - acc: 0.2545 - val_loss: 1.7046 - val_acc: 0.2520
Epoch 3/150
49896/49896 [==============================] - 2s 33us/step - loss: 1.7169 - acc: 0.2625 - val_loss: 1.6987 - val_acc: 0.2341
Epoch 4/150
49896/49896 [==============================] - 2s 33us/step - loss: 1.6617 - acc: 0.2953 - val_loss: 1.5257 - val_acc: 0.3571
Epoch 5/150
49896/49896 [==============================] - 2s 33us/step - loss: 1.4899 - acc: 0.4234 - val_loss: 1.3739 - val_acc: 0.4504
Epoch 6/150
49896/49896 [==============================] - 2s 33us/step - loss: 1.2970 - acc: 0.4683 - val_loss: 1.2223 - val_acc: 0.4067
Epoch 7/150
49896/49896 [==============================] - 2s 32us/step - loss: 1.2120 - acc: 0.4644 - val_loss: 1.1818 - va

49896/49896 [==============================] - 2s 34us/step - loss: 0.5424 - acc: 0.7966 - val_loss: 0.7904 - val_acc: 0.7361
Epoch 60/150
49896/49896 [==============================] - 2s 35us/step - loss: 0.5394 - acc: 0.7979 - val_loss: 0.7693 - val_acc: 0.7361
Epoch 61/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.5387 - acc: 0.7981 - val_loss: 0.8276 - val_acc: 0.7361
Epoch 62/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.5359 - acc: 0.8005 - val_loss: 0.7758 - val_acc: 0.7321
Epoch 63/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.5342 - acc: 0.7989 - val_loss: 0.7576 - val_acc: 0.7361
Epoch 64/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.5263 - acc: 0.8034 - val_loss: 0.7511 - val_acc: 0.7480
Epoch 65/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.5290 - acc: 0.8013 - val_loss: 0.7819 - val_acc: 0.7302
Epoch 66/150
49896/49896 [==============

Epoch 118/150
49896/49896 [==============================] - 2s 35us/step - loss: 0.4245 - acc: 0.8431 - val_loss: 0.8854 - val_acc: 0.7302
Epoch 119/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.4247 - acc: 0.8415 - val_loss: 0.9206 - val_acc: 0.7341
Epoch 120/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.4220 - acc: 0.8415 - val_loss: 0.8699 - val_acc: 0.7639
Epoch 121/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.4231 - acc: 0.8420 - val_loss: 0.8506 - val_acc: 0.7401
Epoch 122/150
49896/49896 [==============================] - 2s 33us/step - loss: 0.4315 - acc: 0.8401 - val_loss: 0.8674 - val_acc: 0.7639
Epoch 123/150
49896/49896 [==============================] - 2s 33us/step - loss: 0.4143 - acc: 0.8438 - val_loss: 0.8692 - val_acc: 0.7440
Epoch 124/150
49896/49896 [==============================] - 2s 34us/step - loss: 0.4231 - acc: 0.8426 - val_loss: 0.8865 - val_acc: 0.7480
Epoch 125/150
49896/

In [28]:
from preprocessing import *
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(256, input_dim=bins * 3 + 9, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(categories, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(XhistPlace, y, epochs=300, batch_size=256, validation_split=0.01)


Train on 49896 samples, validate on 504 samples
Epoch 1/300
49896/49896 [==============================] - 3s 53us/step - loss: 2.2132 - acc: 0.2063 - val_loss: 1.4604 - val_acc: 0.3274
Epoch 2/300
49896/49896 [==============================] - 2s 35us/step - loss: 1.3709 - acc: 0.4130 - val_loss: 1.2021 - val_acc: 0.5298
Epoch 3/300
49896/49896 [==============================] - 2s 36us/step - loss: 1.1784 - acc: 0.5138 - val_loss: 1.1406 - val_acc: 0.5397
Epoch 4/300
49896/49896 [==============================] - 2s 36us/step - loss: 1.0755 - acc: 0.5488 - val_loss: 1.0624 - val_acc: 0.5774
Epoch 5/300
49896/49896 [==============================] - 2s 36us/step - loss: 1.0278 - acc: 0.5673 - val_loss: 0.9645 - val_acc: 0.5913
Epoch 6/300
49896/49896 [==============================] - 2s 36us/step - loss: 0.9831 - acc: 0.5813 - val_loss: 0.9685 - val_acc: 0.5893
Epoch 7/300
49896/49896 [==============================] - 2s 36us/step - loss: 0.9513 - acc: 0.5923 - val_loss: 0.9811 - va

49896/49896 [==============================] - 2s 48us/step - loss: 0.5395 - acc: 0.8020 - val_loss: 0.7780 - val_acc: 0.7520
Epoch 60/300
49896/49896 [==============================] - 3s 54us/step - loss: 0.5328 - acc: 0.8038 - val_loss: 0.7242 - val_acc: 0.7421
Epoch 61/300
49896/49896 [==============================] - 3s 51us/step - loss: 0.5200 - acc: 0.8077 - val_loss: 0.8125 - val_acc: 0.7440
Epoch 62/300
49896/49896 [==============================] - 3s 51us/step - loss: 0.5308 - acc: 0.8071 - val_loss: 0.7824 - val_acc: 0.7440
Epoch 63/300
49896/49896 [==============================] - 2s 43us/step - loss: 0.5163 - acc: 0.8111 - val_loss: 0.7772 - val_acc: 0.7341
Epoch 64/300
49896/49896 [==============================] - 2s 43us/step - loss: 0.5196 - acc: 0.8113 - val_loss: 0.7547 - val_acc: 0.7401
Epoch 65/300
49896/49896 [==============================] - 2s 44us/step - loss: 0.5210 - acc: 0.8078 - val_loss: 0.7553 - val_acc: 0.7520
Epoch 66/300
49896/49896 [==============

49896/49896 [==============================] - 2s 39us/step - loss: 0.4191 - acc: 0.8468 - val_loss: 0.8651 - val_acc: 0.7619
Epoch 118/300
49896/49896 [==============================] - 2s 42us/step - loss: 0.4151 - acc: 0.8482 - val_loss: 0.8678 - val_acc: 0.7639
Epoch 119/300
49896/49896 [==============================] - 2s 38us/step - loss: 0.4187 - acc: 0.8453 - val_loss: 0.9808 - val_acc: 0.7460
Epoch 120/300
49896/49896 [==============================] - 2s 36us/step - loss: 0.4192 - acc: 0.8456 - val_loss: 0.9183 - val_acc: 0.7500
Epoch 121/300
49896/49896 [==============================] - 2s 35us/step - loss: 0.4160 - acc: 0.8478 - val_loss: 0.9600 - val_acc: 0.7262
Epoch 122/300
49896/49896 [==============================] - 2s 37us/step - loss: 0.4091 - acc: 0.8502 - val_loss: 0.9174 - val_acc: 0.7579
Epoch 123/300
49896/49896 [==============================] - 2s 35us/step - loss: 0.4130 - acc: 0.8481 - val_loss: 0.8786 - val_acc: 0.7619
Epoch 124/300
49896/49896 [=======

KeyboardInterrupt: 

In [37]:
from preprocessing import *
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(256, input_dim=bins * 3 + 9, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(categories, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(XhistPlace, y, epochs=100, batch_size=512, validation_split=0.005)


Train on 50148 samples, validate on 252 samples
Epoch 1/100
50148/50148 [==============================] - 2s 44us/step - loss: 8.0306 - acc: 0.4336 - val_loss: 5.5937 - val_acc: 0.5159
Epoch 2/100
50148/50148 [==============================] - 1s 23us/step - loss: 2.9444 - acc: 0.5468 - val_loss: 1.3959 - val_acc: 0.5833
Epoch 3/100
50148/50148 [==============================] - 1s 23us/step - loss: 1.2060 - acc: 0.6241 - val_loss: 1.1110 - val_acc: 0.6429
Epoch 4/100
50148/50148 [==============================] - 1s 24us/step - loss: 0.9733 - acc: 0.6660 - val_loss: 0.9708 - val_acc: 0.6230
Epoch 5/100
50148/50148 [==============================] - 1s 24us/step - loss: 0.8872 - acc: 0.6841 - val_loss: 0.9440 - val_acc: 0.6468
Epoch 6/100
50148/50148 [==============================] - 1s 25us/step - loss: 0.8379 - acc: 0.6999 - val_loss: 0.8568 - val_acc: 0.6706
Epoch 7/100
50148/50148 [==============================] - 1s 27us/step - loss: 0.8050 - acc: 0.7068 - val_loss: 0.9378 - va

50148/50148 [==============================] - 1s 26us/step - loss: 0.4974 - acc: 0.8139 - val_loss: 0.8337 - val_acc: 0.7381
Epoch 60/100
50148/50148 [==============================] - 1s 26us/step - loss: 0.5033 - acc: 0.8112 - val_loss: 0.8574 - val_acc: 0.7222
Epoch 61/100
50148/50148 [==============================] - 1s 25us/step - loss: 0.5010 - acc: 0.8130 - val_loss: 0.8574 - val_acc: 0.7222
Epoch 62/100
50148/50148 [==============================] - 1s 25us/step - loss: 0.4983 - acc: 0.8144 - val_loss: 1.0042 - val_acc: 0.6865
Epoch 63/100
50148/50148 [==============================] - 1s 25us/step - loss: 0.4998 - acc: 0.8095 - val_loss: 0.7978 - val_acc: 0.7381
Epoch 64/100
50148/50148 [==============================] - 1s 26us/step - loss: 0.4948 - acc: 0.8147 - val_loss: 0.8697 - val_acc: 0.7143
Epoch 65/100
50148/50148 [==============================] - 1s 25us/step - loss: 0.4982 - acc: 0.8134 - val_loss: 0.8389 - val_acc: 0.7381
Epoch 66/100
50148/50148 [==============

In [42]:
XhistPlace_test = np.hstack((Xtest_hist, testPlaces))
predict = model.predict(XhistPlace_test)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest.argmax(axis=1), predict.argmax(axis=1)))
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest.argmax(axis=1), predict.argmax(axis=1)))
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(ytest.argmax(axis=1), predict.argmax(axis=1))

[[1012  190   18  103  334  108   35]
 [ 143 1422    5  121   57   37   15]
 [   8    1 1769    0    3    0   19]
 [ 127  232    7 1105  291   24   14]
 [ 141   70    9  170 1351   24   35]
 [  50   29    1    8    8 1704    0]
 [  30   19   52   11   53    0 1635]]
0.7934920634920635


(array([0.66975513, 0.72440143, 0.95056421, 0.72793149, 0.6442537 ,
        0.89826041, 0.93268682]),
 array([0.56222222, 0.79      , 0.98277778, 0.61388889, 0.75055556,
        0.94666667, 0.90833333]),
 array([0.61129568, 0.75577996, 0.96640262, 0.66606389, 0.69335386,
        0.92182851, 0.920349  ]),
 array([1800, 1800, 1800, 1800, 1800, 1800, 1800], dtype=int64))

In [46]:
from preprocessing import *
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(32, input_dim=bins * 3 + 9, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(categories, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(XhistPlace, y, epochs=100, batch_size=128, validation_split=0.005)

Train on 50148 samples, validate on 252 samples
Epoch 1/100
50148/50148 [==============================] - 4s 86us/step - loss: 1.7403 - acc: 0.3953 - val_loss: 1.1935 - val_acc: 0.5159
Epoch 2/100
50148/50148 [==============================] - 3s 59us/step - loss: 1.1599 - acc: 0.5398 - val_loss: 1.0075 - val_acc: 0.5675
Epoch 3/100
50148/50148 [==============================] - 3s 59us/step - loss: 1.0348 - acc: 0.5899 - val_loss: 0.9631 - val_acc: 0.6230
Epoch 4/100
50148/50148 [==============================] - 3s 62us/step - loss: 0.9752 - acc: 0.6185 - val_loss: 0.9273 - val_acc: 0.6587
Epoch 5/100
50148/50148 [==============================] - 3s 61us/step - loss: 0.9357 - acc: 0.6429 - val_loss: 0.8153 - val_acc: 0.7103
Epoch 6/100
50148/50148 [==============================] - 3s 64us/step - loss: 0.9004 - acc: 0.6570 - val_loss: 0.8558 - val_acc: 0.6587
Epoch 7/100
50148/50148 [==============================] - 3s 65us/step - loss: 0.8649 - acc: 0.6735 - val_loss: 0.8217 - va

50148/50148 [==============================] - 3s 65us/step - loss: 0.6341 - acc: 0.7652 - val_loss: 0.7000 - val_acc: 0.7302
Epoch 60/100
50148/50148 [==============================] - 3s 65us/step - loss: 0.6354 - acc: 0.7641 - val_loss: 0.7230 - val_acc: 0.7222
Epoch 61/100
50148/50148 [==============================] - 3s 66us/step - loss: 0.6388 - acc: 0.7630 - val_loss: 0.7321 - val_acc: 0.7222
Epoch 62/100
50148/50148 [==============================] - 3s 65us/step - loss: 0.6365 - acc: 0.7624 - val_loss: 0.7717 - val_acc: 0.7103
Epoch 63/100
50148/50148 [==============================] - 3s 65us/step - loss: 0.6357 - acc: 0.7626 - val_loss: 0.7420 - val_acc: 0.7103
Epoch 64/100
50148/50148 [==============================] - 3s 67us/step - loss: 0.6330 - acc: 0.7639 - val_loss: 0.7207 - val_acc: 0.7103
Epoch 65/100
50148/50148 [==============================] - 3s 66us/step - loss: 0.6316 - acc: 0.7655 - val_loss: 0.7432 - val_acc: 0.7222
Epoch 66/100
50148/50148 [==============

In [47]:
XhistPlace_test = np.hstack((Xtest_hist, testPlaces))
predict = model.predict(XhistPlace_test)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest.argmax(axis=1), predict.argmax(axis=1)))
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest.argmax(axis=1), predict.argmax(axis=1)))
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(ytest.argmax(axis=1), predict.argmax(axis=1))

[[1037  101   26  144  315  137   40]
 [ 188 1139    5  319   81   50   18]
 [   8    0 1780    0    0    0   12]
 [ 110  135    7 1184  325   23   16]
 [ 173   30   12  162 1357   45   21]
 [  51   10    1    6   13 1718    1]
 [  40    5   68   12  104    0 1571]]
0.7766666666666666


(array([0.6453018 , 0.80211268, 0.93733544, 0.64805692, 0.61822323,
        0.8707552 , 0.935676  ]),
 array([0.57611111, 0.63277778, 0.98888889, 0.65777778, 0.75388889,
        0.95444444, 0.87277778]),
 array([0.6087467 , 0.70745342, 0.96242228, 0.65288117, 0.67934919,
        0.91068116, 0.90313308]),
 array([1800, 1800, 1800, 1800, 1800, 1800, 1800], dtype=int64))

In [ ]:
from preprocessing import *
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(256, input_dim=bins * 3 + 9, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(categories, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(XhistPlace, y, epochs=100, batch_size=512, validation_split=0.005)
